In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-125729")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-125729
Azure region: southcentralus
Subscription id: 374bdf1a-c648-4244-a317-f0d1ef4b85c7
Resource group: aml-quickstarts-125729


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cpu_cluster_name='cpucluster'

compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    
"--C": choice(0.01,0.03,0.05,0.1) , 
"--max_iter": choice(range(50, 200)) })

# Specify a Policy
policy =  BanditPolicy(slack_factor = 0.1, evaluation_interval=3, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory =
 '.', entry_script='train.py', 
   compute_target='cpucluster')


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig (estimator=est,
                             hyperparameter_sampling=ps,
                             policy= policy,
                             primary_metric_name='Accuracy',
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=15,
                             max_concurrent_runs=4)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.widgets import RunDetails

hyperdrive_run = exp.submit(hyperdrive_config)

RunDetails(hyperdrive_run).show()

run.wait_for_completion(show_output=True)

hyperdrive_run.get_status()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: 9c53e9bc-a989-413a-8f9c-98f6f635f85b
Web View: https://ml.azure.com/experiments/udacity-project/runs/9c53e9bc-a989-413a-8f9c-98f6f635f85b?wsid=/subscriptions/374bdf1a-c648-4244-a317-f0d1ef4b85c7/resourcegroups/aml-quickstarts-125729/workspaces/quick-starts-ws-125729


In [12]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics=best_run.get_metrics()

print(best_run.get_details()['runDefinition']['arguments'])
 
best_run.download_file("outputs/model.joblib","./outputs/model.joblib")
model = best_run.register_model(model_name='hyperdrive_model', model_path='outputs/model.joblib')


print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])



['--C', '0.1', '--max_iter', '141']
Best Run Id:  HD_7259332b-94c3-4f32-becc-8b13f7ef16f3_8

 Accuracy: 0.9171471927162367


In [13]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds_auto =TabularDatasetFactory.from_delimited_files(['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'])


In [44]:
from train import clean_data
from sklearn.model_selection import train_test_split


# Use the clean_data function to clean your data.
x, y = clean_data(ds_auto)


from sklearn import datasets
from azureml.core.dataset import Dataset
from scipy import sparse
import numpy as np
import pandas as pd

# Create a project_folder if it doesn't exist
    
if "data" not in os.listdir():
    os.mkdir("./data")

X = pd.DataFrame(x)
y = pd.DataFrame(y)

# merge X and y
label = "y"
x[label] = y

training_data = x

train_data, test_data = train_test_split(training_data, test_size=0.3)


train_data.to_csv('data/autodata.csv')
ds = ws.get_default_datastore()
ds.upload(src_dir='./data', target_path='automldata', overwrite=True, show_progress=True)

train_data = Dataset.Tabular.from_delimited_files(path=ds.path('automldata/autodata.csv'))


In [41]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    compute_target='cpucluster',
    training_data=train_data,
    label_column_name=label,
    n_cross_validations=5)

In [42]:
# Submit your automl run

remote_run = exp.submit(automl_config, show_output=True)



Running on remote.
Running on remote compute: cpucluster
Parent Run ID: AutoML_d0d46f3e-907e-492d-90b4-e60cc9a1c7f5

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the s

In [43]:
# Retrieve and save your best automl model.
remote_run.wait_for_completion()


best_run, fitted_model = remote_run.get_output()

model_name = best_run.properties['automl']

script_file_name = 'inference/score.py'

best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')


KeyboardInterrupt: 